In [1]:
from pathlib import Path
import json
from collections import defaultdict

In [12]:
import cv2
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [7]:
# data_dir = Path(r"C:\datasets\pubtabnet")
data_dir = Path(r"C:\Users\stans\Documents\Projects\Datasets\pubtabnet.tar\pubtabnet\pubtabnet")
test_dir = data_dir / "train"

In [8]:
files = list(test_dir.glob("*.png"))

500777

In [9]:
with open(data_dir / "PubTabNet_2.0.0.jsonl", "r", encoding="utf8") as f:
    json_list = list(f)

In [10]:
fname_index_map = {}

for i, json_str in enumerate(json_list):
    result = json.loads(json_str)
    fname_index_map[result["filename"]] = i

## Data Utils
Utility functions for interacting with the PubTabNet Dataset

In [11]:
def Y(i):
    fname = files[i]
    i = fname_index_map[fname.parts[-1]]
    return json.loads(json_list[i])

def X(i):
    return Image.open(files[i])

In [74]:
def clean_tokens(tokens):
    clean = []
    for j in range(len(tokens)):
        if tokens[j] == "<td":
            clean.append(int(tokens[j+1].split('"')[1]))  # colspan="x"
        elif tokens[j] == "<td>":
            clean.append(1)
    return clean

In [151]:
class ImageLoader(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, files):
        'Initialization'
        self.files = files

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.files)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        f = files[index]
        img = np.array(Image.open(f))
        return img

In [152]:
class TargetLoader(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, files):
        'Initialization'
        self.files = files

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.files)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        f = files[index]
        i = fname_index_map[f.parts[-1]]
        data = json.loads(json_list[i])
        d = {}
        d['boxes'] = torch.as_tensor([np.array(x.get("bbox",[0,0,0,0])) for x in data['html']['cells'] if 'bbox' in x], dtype=torch.float32)
        d['labels'] = torch.tensor(clean_tokens(data['html']['structure']['tokens']), dtype=torch.int64)
        return d

In [160]:
train = files[:1000]
test = files[1000:2000]

In [165]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
output = model(ImageLoader(train), TargetLoader(train))
model.train()

KeyboardInterrupt: 